# Day 6

The sea floor is getting steeper. Maybe the sleigh keys got carried this way?

A massive school of glowing lanternfish swims past. They must spawn quickly to reach such large numbers - maybe exponentially quickly? You should model their growth rate to be sure.

Although you know nothing about this specific species of lanternfish, you make some guesses about their attributes. Surely, each lanternfish creates a new lanternfish once every 7 days.

However, this process isn't necessarily synchronized between every lanternfish - one lanternfish might have 2 days left until it creates another lanternfish, while another might have 4. So, you can model each fish as a single number that represents the number of days until it creates a new lanternfish.

Furthermore, you reason, a new lanternfish would surely need slightly longer before it's capable of producing more lanternfish: two more days for its first cycle.

So, suppose you have a lanternfish with an internal timer value of 3:

After one day, its internal timer would become 2.
After another day, its internal timer would become 1.
After another day, its internal timer would become 0.
After another day, its internal timer would reset to 6, and it would create a new lanternfish with an internal timer of 8.
After another day, the first lanternfish would have an internal timer of 5, and the second lanternfish would have an internal timer of 7.
A lanternfish that creates a new fish resets its timer to 6, not 7 (because 0 is included as a valid timer value). The new lanternfish starts with an internal timer of 8 and does not start counting down until the next day.

Realizing what you're trying to do, the submarine automatically produces a list of the ages of several hundred nearby lanternfish (your puzzle input). For example, suppose you were given the following list:

3,4,3,1,2
This list means that the first fish has an internal timer of 3, the second fish has an internal timer of 4, and so on until the fifth fish, which has an internal timer of 2. Simulating these fish over several days would proceed as follows:

```
Initial state: 3,4,3,1,2
After  1 day:  2,3,2,0,1
After  2 days: 1,2,1,6,0,8
After  3 days: 0,1,0,5,6,7,8
After  4 days: 6,0,6,4,5,6,7,8,8
After  5 days: 5,6,5,3,4,5,6,7,7,8
After  6 days: 4,5,4,2,3,4,5,6,6,7
After  7 days: 3,4,3,1,2,3,4,5,5,6
After  8 days: 2,3,2,0,1,2,3,4,4,5
After  9 days: 1,2,1,6,0,1,2,3,3,4,8
After 10 days: 0,1,0,5,6,0,1,2,2,3,7,8
After 11 days: 6,0,6,4,5,6,0,1,1,2,6,7,8,8,8
After 12 days: 5,6,5,3,4,5,6,0,0,1,5,6,7,7,7,8,8
After 13 days: 4,5,4,2,3,4,5,6,6,0,4,5,6,6,6,7,7,8,8
After 14 days: 3,4,3,1,2,3,4,5,5,6,3,4,5,5,5,6,6,7,7,8
After 15 days: 2,3,2,0,1,2,3,4,4,5,2,3,4,4,4,5,5,6,6,7
After 16 days: 1,2,1,6,0,1,2,3,3,4,1,2,3,3,3,4,4,5,5,6,8
After 17 days: 0,1,0,5,6,0,1,2,2,3,0,1,2,2,2,3,3,4,4,5,7,8
After 18 days: 6,0,6,4,5,6,0,1,1,2,6,0,1,1,1,2,2,3,3,4,6,7,8,8,8,8
Each day, a 0 becomes a 6 and adds a new 8 to the end of the list, while each other number decreases by 1 if it was present at the start of the day.
```

In this example, after 18 days, there are a total of 26 fish. After 80 days, there would be a total of 5934.

In [1]:
sample_input = [3, 4, 3, 1, 2]

with open("data/day-6.txt") as file:
    lines = [[int(i) for i in line.strip().split(",")] for line in file.readlines()][0]

lines[:5]

[1, 3, 4, 1, 1]

## Part 1

Find a way to simulate lanternfish. How many lanternfish would there be after 80 days?

In [2]:
from tqdm.notebook import tqdm


class Fish:
    def __init__(self, timer):
        self.timer = timer

    def age(self):
        self.timer -= 1

    @classmethod
    def spawn(cls):
        return cls(timer=8)


def run_simulation(school, num_days):
    """Run a fish spawning simulation."""
    for i in tqdm(range(num_days)):

        babies = []
        for fish in school:

            if fish.timer == 0:
                fish.timer = 6
                new_fish = fish.spawn()
                babies.append(new_fish)
            else:
                fish.age()

        school.extend(babies)

    return school


test_school = [Fish(i) for i in sample_input]


test_total = run_simulation(test_school, num_days=18)
test_population = len(test_total)
assert test_population == 26

  0%|          | 0/18 [00:00<?, ?it/s]

In [3]:
school = [Fish(line) for line in lines]

total = run_simulation(school, num_days=80)
population = len(total)
population

  0%|          | 0/80 [00:00<?, ?it/s]

390011

## Part 2

Suppose the lanternfish live forever and have unlimited food and space. Would they take over the entire ocean?

After 256 days in the example above, there would be a total of `26984457539` lanternfish!

How many lanternfish would there be after 256 days?



### Notes (Thought Process)
- The solution for Part 1 doesn't work because we would quickly run out of memory if storing all the fish.
- So how do we get an accurate count without storing all the fish?
- Thus far a running sum seems like a good solution. 
- We know it takes 8 days for each fish to reproduce. 
- Can we keep track of the number of fish and the internal states without actually having all of the fish?
- If we hold the number of fish at each state in a cohort, we know when that cohort will reproduce. We can also keep track of the aging by moving the fish counts down one each day. 

In [25]:
school = {
    0: 0,
    1: 0,
    2: 0,
    3: 0,
    4: 0,
    5: 0,
    6: 0,
    7: 0,
}

def swap(school, indices):
    one, two = indices
    temp = school[one]
    school[one] = school[two]
    school[two] = temp
    return school

for line in sample_input:
    school[line]+=1



for i in range(18):    
    
    if school.get(0):
        school[6
    school[0] = school.pop(6)
    


{0: 0, 1: 1, 2: 1, 3: 2, 4: 1, 5: 0, 6: 0, 7: 0}
{0: 0, 1: 2, 2: 1, 3: 1, 4: 1, 5: 0, 6: 0, 7: 0}
